# Analiza Wielowymiarowa - zajecia 2 - Statystyczne podstawy

In [ ]:
from multidim.utils import resolve_stata, load_stata

STATA_PATH, STATA_TYPE = resolve_stata(version = 18, stype = "se")
# make sure they are proper ones
STATA_PATH, STATA_TYPE

In [ ]:
load_stata(STATA_PATH, STATA_TYPE)

In [ ]:
from scipy.stats import norm, t, f, chi2, describe, iqr
from numpy import quantile, linspace, std, mean
import matplotlib.pyplot as plt

### Uzycie staty w jupyterach

%stata dla wywolan jednoliniowych
%%stata dla wywolan wieloliniowych

oba wywolania posiadaja dodatkowe argumenty ktore omowimy podczas zajec
%%stata -d dane -eret myeret

Obiekt stata z cala gama metod takich jak stata.run

### Rozpoczynamy wywolywanie Stata poprzez jupyter

ustawienie liczby obserwacji w zbiorze

In [ ]:
%stata set obs 10000

 ustawienie generatora liczb pseudolosowych

In [ ]:
%stata set seed 1234

In [ ]:
# np.random.seed(1234)
from numpy.random import MT19937
from numpy.random import RandomState, SeedSequence
rs = RandomState(MT19937(SeedSequence(1234)))

 wylosowanie liczb losowych z rozkladu jednostajnego, opcja double nakazuje wykonanie operacji o podwojonej precyzji numerycznej

In [ ]:
%stata gen double u=uniform()

In [ ]:
u = rs.uniform(size = 1000)

 przeksztalcenie zmiennej u na zmienną o rozkladzie normalnym

In [ ]:
%stata gen double x1 = invnorm(u)

In [ ]:
x1 = norm.ppf(u)

 histogram

In [ ]:
%stata histogram x1

In [ ]:
_ = plt.hist(x1, bins='auto')
plt.show()

 szczegółowy opis statystyczny zmiennej

In [ ]:
%stata su x1, det

In [ ]:
describe(x1)

 przekształcenie zmiennej u na zmienną o rozkładzie t-Studenta o 5 stopniach swobowy

In [ ]:
%stata gen double x2 = invt(5,u) 

In [ ]:
x2 = t.ppf(u, 5)

 histogram

In [ ]:
%stata hist x2, norm

In [ ]:
plt.hist(x2, bins='auto', density=True)
xmin, xmax = plt.xlim()
x = linspace(xmin, xmax, 100)
p = norm.pdf(x, mean(x2), std(x2))
plt.plot(x, p, 'k', linewidth=2)
plt.show()

 szczegółowy opis statystyczny zmiennej

In [ ]:
%stata su x2, det

In [ ]:
describe(x2)

 przekształcenie zmiennej u na zmienną o rozkładzie chi-kwadrat o 2 stopniach swobody

In [ ]:
%stata gen double x3 = invchi2(2,u) 

In [ ]:
x3 = chi2.ppf(u, 2)

 histogram

In [ ]:
%stata hist x3, norm

In [ ]:
_ = plt.hist(x3, bins='auto')
plt.show()

 szczegółowy opis statystyczny zmiennej

In [ ]:
%stata su x3, det

In [ ]:
describe(x3)

 przekształcenie zmiennej u na zmienną o rozkładzie F o 2,10 stopniach swobody

In [ ]:
%stata gen double x4 = invF(2,10,u) 

In [ ]:
x4 = f.ppf(u,2,10)

 histogram

In [ ]:
%stata hist x4, norm

In [ ]:
_ = plt.hist(x3, bins='auto')
plt.show()

 szczegółowy opis statystyczny zmiennej

In [ ]:
%stata su x4, det

In [ ]:
describe(x4)

## STATYSTYKI OPISOWE 

In [ ]:
%stata sysuse auto, clear

In [ ]:
from multidim.datasets import load_auto
auto = load_auto()
# auto = stata.pdataframe_from_data(missingval = np.NaN, valuelabel = True)

 kwantyle

In [ ]:
%stata tabstat price, stat(q)

In [ ]:
# 8 different methods
quantile(auto.price, (0.25, 0.5, 0.75), method="averaged_inverted_cdf")

 odstep miedzykwantylowy

In [ ]:
%stata tabstat price, stat(iqr)

In [ ]:
# many different interpolation
iqr(auto.price, interpolation = "nearest")

 wykres pudelkowy

 dolna i górna krawędż pudełka to odpowiednio wartość pierwszego i trzeciego kwartyla, linia w poprzek pudełka prezentuje wartość medany, a "wasy" pokazują 95% przedział ufności

In [ ]:
%stata graph box price

In [ ]:
auto.price.plot.box()

 histogram, opcja bin określa liczbę prostokątów, opcja freq przelicza gestosc na częstości

In [ ]:
%stata histogram price, bin(15)

In [ ]:
auto.price.plot.hist()

In [ ]:
%stata histogram price, bin(15) freq

In [ ]:
auto.price.plot.hist(bins=15)

 estymator jądrowy

 o wygładzeniu rozkładu decyduje opcja bwidth - szerokosc pasma

In [ ]:
%stata kdensity price

In [ ]:
auto.price.plot.kde()

In [ ]:
%stata kdensity price, bwidth(1200)

In [ ]:
auto.price.plot.kde(ind=300)

In [ ]:
%stata kdensity price, bwidth(10000)

In [ ]:
auto.price.plot.kde(ind=3)

 wyczyszczenie pamieci

In [ ]:
%stata clear

 zaladowanie zbioru danych auto dostarczonych z pakietem Stata

In [ ]:
%stata sysuse auto

 jednowymiarowa tabela czestosci

In [ ]:
%stata tab foreign

In [ ]:
# [[ (not [) to persist the DataFrame
res = auto.groupby("foreign")[['price']].count()
res["Percent"] = (res / res.sum()) * 100
res["Cum"] = res["Percent"].cumsum()
res